## Problem 1 - Solution

### Import necessary libraries

In [1]:
%reset

Once deleted, variables cannot be recovered. Proceed (y/[n])? y


In [2]:
import urllib #for web scraping
from bs4 import BeautifulSoup #for web scraping
from pandas.io.parsers import TextParser #used in web scraping
from lxml.html import parse #used in web scraping
from urllib.request import urlopen #for writing csv
import csv #for writing csv
import pandas as pd #for dataframes
import logging #for logging

logging.basicConfig(filename='problem1_log.log', filemode='w', format='%(asctime)s %(message)s', datefmt='%m/%d/%Y %I:%M:%S %p', level=logging.DEBUG)
logging.info('Program Starting...')
#logger = logging.getLogger()
#logger.setLevel(logging.DEBUG)

### Function to fetch the first URL by passing CIK and ACC_No
- CIK and Acc_no are passed into a function. \n
- Leading zeroes are removed from CIK.
- Generate the URL
- Handles the exception when no input is given 

In [3]:
url1 = ""
url_pre = "https://www.sec.gov/Archives/edgar/data/"

def geturl(cik='', accno=''):
    if not cik or not accno:
        logging.warning('CIK or AccNo was not given, assuming it to be 0000051143 and 0000051143-13-000007 respectively')
        cik='0000051143'
        accno = '0000051143-13-000007'
        cik_striped = cik.lstrip("0") #strip leading zeroes
        accno_striped = accno.replace("-","")
        url1 = url_pre + cik_striped + "/" + accno_striped + "/" + accno + "-index.html"
        return url1
    else:
        cik_striped = cik.lstrip("0") #strip leading zeroes
        accno_striped = accno.replace("-","")
        url1 = url_pre + cik_striped + "/" + accno_striped + "/" + accno + "-index.html"
        return url1

In [6]:
geturl("0000051143", "0000051143-13-000007")

'https://www.sec.gov/Archives/edgar/data/51143/000005114313000007/0000051143-13-000007-index.html'

### Function to fetch the form's URL

In [5]:
#page = urllib.request.urlopen(geturl("0000051143", "0000051143-13-000007"))
try:
    page = urllib.request.urlopen(geturl("0000051143", "0000051143-13-000007"))
    
    # parse the page and save it in soup format 
    soup = BeautifulSoup(page,"lxml")

    # find the form name according to accession no
    form = soup.find(id='formName').get_text()
    formname = form[6:10]

    # find the td which has the above form name
    formtype = soup.findAll('td', text = formname)[0]

    # fetch the url for that form.. Argh!!! find all siblings and then child href ? maybe
    #formprevious = formtype.findAllPrevious
    #print(formprevious)

    url2 = ""
    all_links = soup.find_all('a')
    for link in all_links:
        if "10q.htm" in link.get("href"):
            url2 = "https://www.sec.gov/" + link.get("href")

except urllib.error.HTTPError as err:
    print("Invalid CIK or AccNo")
    logging.warning("Invalid CIK or AccNo")
    
print(url2)

https://www.sec.gov//Archives/edgar/data/51143/000005114313000007/ibm13q3_10q.htm


### Access the Form and fetch the tables

In [65]:
# retrieve the web page
print(url2)
page_10q = urllib.request.urlopen(url2)
soup = BeautifulSoup(page_10q,"lxml")

https://www.sec.gov//Archives/edgar/data/51143/000005114313000007/ibm13q3_10q.htm


In [6]:
all_tables= soup.select('div["bclpageborder"] table')
#all_tables = soup.findAll('table')
for tab in all_tables:
    records = []
    for tr in tab.find_all('tr'):
        rowString=[]
        for td in tr.findAll('td'):
            p = td.find_all('p')
            if len(p)>0:
                for ps in p:
                    ps_text = ps.get_text().replace("\n"," ") 
                    ps_text = ps_text.replace("\xa0","")                 
                    rowString.append(ps_text)
            else:
                td_text=td.get_text().replace("\n"," ")
                td_text = td_text.replace("\xa0","")
                rowString.append(td_text)
        records.append(rowString)        
    with open(str(all_tables.index(tab)) + 'tables.csv', 'w') as f:
        writer = csv.writer(f)
        writer.writerows(records)
    